In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
import prettytable as pt
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")



###################################################################################################
                                                                                                  #
from rhoova.Client import *                                                                       #   
#Register and get api key from https://app.rhoova.com/ for ClientConfig("api key", "api secret")  #
config = ClientConfig("", "")                #                                                    #
api = Api(config)                                                                                 # 
                                                                                                  #
###################################################################################################


directory = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
yielddatadirectory=directory+"/data/yielddata/yielddata.csv"

yielddata = pd.read_csv(yielddatadirectory)
yielddata = yielddata.replace(np.nan, '', regex=True) 

In [2]:
data = {
  "notional": 1000,
  "valuationDate":"2021-01-28",
  "settlementDate":"2021-02-01",
  "buySell": "Buy",
  "floatingBondDefinition": {
    "calendar": "Turkey",
    "spread": 0.0,
    "currency": "TRY",
    "dateGeneration": "Backward",
    "dayCounter": "Actual360",
    "endOfMonth": True,
    "frequency": "Quarterly",
    "issueDate":  "2015-07-15",
    "maturityDate": "2025-07-15",
    #"fixingDate":["2015-07-13","2015-10-13","2016-01-13","2016-04-13","2016-07-13","2016-10-13",
    #              "2017-01-12","2017-04-13","2017-07-13","2017-10-12","2018-01-11","2018-04-12",
    #              "2018-07-12","2018-10-11","2019-01-11","2019-04-11","2019-07-11","2019-10-11",
    #              "2020-01-13","2020-04-13","2020-07-13","2020-10-13","2021-01-13"],
    #"fixingRate":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001817],
    "fixingDate":[],
    "fixingRate":[],
    "businessDayConvention": "ModifiedFollowing",
    "maturityDateConvention": "ModifiedFollowing",
    "redemption": 100
  },
  "discountCurve": {
    "settlementDays": 2,
    "intpMethod": "LINEAR",
    "currency": "TRY",
    "calendar": "Turkey",
    "dayCounter": "Actual360",
    "period": "6M",
    "instruments": {
      "IRS": {
        "frequency": "Semiannual",
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      },
      "DEPO": {
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      }
    }
  },
  "yieldData": yielddata.to_dict('r')
}


In [3]:
try:
    data = api.createTask(CalculationType.FLOATING_RATE_BOND, data,True)
    result=json.loads(data["result"])
except RhoovaError as e:
    e.printPretty()

In [4]:
npvTable = pt.PrettyTable(['Parameters', 'Value'])
npvTable.add_row(['PV', result.get('pv')])
npvTable.add_row(['Clean Price', result.get('cleanPrice')])
npvTable.add_row(['Dirty Price', result.get('dirtyPrice')])
npvTable.add_row(['Accrued Amount', result.get('accruedAmount')])
npvTable.add_row(['Yield to Maturity', 100*result.get('yieldToMaturity')])
npvTable.add_row(['Duration', result.get('duration')])
npvTable.add_row(['Modified Duration', result.get('modifiedDuration')])
npvTable.add_row(['Macualay Duration', result.get('macaulayDuration')])
npvTable.add_row(['Convexity', result.get('convexity')])
npvTable.add_row(['Bps', result.get('bps')])
npvTable.align = 'r'
#npvTable.float_format = '.4'
print(npvTable)



+-------------------+---------------------+
|        Parameters |               Value |
+-------------------+---------------------+
|                PV |   999.9999999999999 |
|       Clean Price |   99.99999999999999 |
|       Dirty Price |   99.99999999999999 |
|    Accrued Amount |                   0 |
| Yield to Maturity |  0.4345369451045994 |
|          Duration |    4.48857062459583 |
| Modified Duration |   4.483699791570343 |
| Macualay Duration |    4.48857062459583 |
|         Convexity |  21.295141419340162 |
|               Bps | 0.04470213361548963 |
+-------------------+---------------------+


In [5]:
cashflows=pd.DataFrame(result.get("data"))
cashflows=cashflows[cashflows["termToMatByDay"]>0]
cashflows

,fixingDate,accrualStart,accrualEnd,notional,currency,leg,payOrReceive,instrument,rate,zeroRate,spread,termToMatByDay,termToMatByYear,cashflow,discountFactor,cashflowPv
0,2021-01-28,2021-02-01,2021-04-15,1000,TRY,Floating,Buy,Floating Rate Bond,0.001817,0.001817,0,73,0.202778,0.368416,0.999632,0.368280
1,2021-04-13,2021-04-15,2021-07-16,1000,TRY,Floating,Buy,Floating Rate Bond,0.002585,0.002245,0,165,0.458333,0.660715,0.998972,0.660036
2,2021-07-13,2021-07-16,2021-10-15,1000,TRY,Floating,Buy,Floating Rate Bond,0.003286,0.002615,0,256,0.711111,0.830713,0.998143,0.829170
3,2021-10-13,2021-10-15,2022-01-17,1000,TRY,Floating,Buy,Floating Rate Bond,0.004224,0.003046,0,350,0.972222,1.102970,0.997043,1.099708
4,2022-01-13,2022-01-17,2022-04-15,1000,TRY,Floating,Buy,Floating Rate Bond,0.002271,0.002890,0,438,1.216667,0.555045,0.996490,0.553097
5,2022-04-13,2022-04-15,2022-07-18,1000,TRY,Floating,Buy,Floating Rate Bond,0.001252,0.002601,0,532,1.477778,0.326903,0.996164,0.325649
6,2022-07-13,2022-07-18,2022-10-17,1000,TRY,Floating,Buy,Floating Rate Bond,0.000682,0.002320,0,623,1.730556,0.172408,0.995992,0.171717
7,2022-10-13,2022-10-17,2023-01-16,1000,TRY,Floating,Buy,Floating Rate Bond,0.000121,0.002040,0,714,1.983333,0.030703,0.995962,0.030579
8,2023-01-12,2023-01-16,2023-04-17,1000,TRY,Floating,Buy,Floating Rate Bond,0.002725,0.002118,0,805,2.236111,0.688854,0.995276,0.685600
9,2023-04-13,2023-04-17,2023-07-17,1000,TRY,Floating,Buy,Floating Rate Bond,0.003632,0.002271,0,896,2.488889,0.917994,0.994363,0.912820


In [6]:
data2 = {
  "notional": 1000,
  "valuationDate":"2021-01-28",
  "settlementDate":"2021-02-01",
  "buySell": "Buy",
  "floatingBondDefinition": {
    "calendar": "Turkey",
    "spread": 0.0,
    "currency": "TRY",
    "dateGeneration": "Backward",
    "dayCounter": "Actual360",
    "endOfMonth": True,
    "frequency": "Quarterly",
    "issueDate":  "2015-07-15",
    "maturityDate": "2025-07-15",
    "fixingDate":["2015-07-13","2015-10-13","2016-01-13","2016-04-13","2016-07-13","2016-10-13",
                  "2017-01-12","2017-04-13","2017-07-13","2017-10-12","2018-01-11","2018-04-12",
                  "2018-07-12","2018-10-11","2019-01-11","2019-04-11","2019-07-11","2019-10-11",
                  "2020-01-13","2020-04-13","2020-07-13","2020-10-13","2021-01-13"],
    "fixingRate":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001817],
    "businessDayConvention": "ModifiedFollowing",
    "maturityDateConvention": "ModifiedFollowing",
    "redemption": 100
  },
  "discountCurve": {
    "applyShock":{
           "method": "parallel",
          "shockValues" : [
        {
          "tenor" : "all",
          "shockValue" : -1.0
        }
      ]
    },
    "settlementDays": 2,
    "intpMethod": "LINEAR",
    "currency": "TRY",
    "calendar": "Turkey",
    "dayCounter": "Actual360",
    "period": "6M",
    "instruments": {
      "IRS": {
        "frequency": "Semiannual",
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      },
      "DEPO": {
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      }
    }
  },
  "yieldData": yielddata.to_dict('r')
}



In [7]:
try:
    data = api.createTask(CalculationType.FLOATING_RATE_BOND, data2,True)
    result2=json.loads(data["result"])
except RhoovaError as e:
    e.printPretty()

In [8]:
npvTable = pt.PrettyTable(['Parameters', 'Value'])
npvTable.add_row(['PV', result2.get('pv')])
npvTable.add_row(['Clean Price', result2.get('cleanPrice')])
npvTable.add_row(['Dirty Price', result2.get('dirtyPrice')])
npvTable.add_row(['Accrued Amount', result2.get('accruedAmount')])
npvTable.add_row(['Yield to Maturity', 100*result2.get('yieldToMaturity')])
npvTable.add_row(['Duration', result2.get('duration')])
npvTable.add_row(['Modified Duration', result2.get('modifiedDuration')])
npvTable.add_row(['Macualay Duration', result2.get('macaulayDuration')])
npvTable.add_row(['Convexity', result2.get('convexity')])
npvTable.add_row(['Bps', result2.get('bps')])
npvTable.align = 'r'
#npvTable.float_format = '.4'
print(npvTable)

+-------------------+----------------------+
|        Parameters |                Value |
+-------------------+----------------------+
|                PV |   1000.1060820046739 |
|       Clean Price |   100.00202792268962 |
|       Dirty Price |    100.0106082004674 |
|    Accrued Amount | 0.008580277777777777 |
| Yield to Maturity |   0.4245542576313025 |
|          Duration |    4.489068217390839 |
| Modified Duration |    4.484308636580357 |
| Macualay Duration |    4.489068217390839 |
|         Convexity |   21.299980859640204 |
|               Bps |  0.04518454597195337 |
+-------------------+----------------------+


In [9]:
data3 = {
  "notional": 1000,
  "valuationDate":"2021-01-28",
  "settlementDate":"2021-02-01",
  "buySell": "Buy",
  "floatingBondDefinition": {
    "calendar": "Turkey",
    "spread": 0.0,
    "currency": "TRY",
    "dateGeneration": "Backward",
    "dayCounter": "Actual360",
    "endOfMonth": True,
    "frequency": "Quarterly",
    "issueDate":  "2015-07-15",
    "maturityDate": "2025-07-15",
   # "fixingDate":["2015-07-13","2015-10-13","2016-01-13","2016-04-13","2016-07-13","2016-10-13",
   #               "2017-01-12","2017-04-13","2017-07-13","2017-10-12","2018-01-11","2018-04-12",
   #               "2018-07-12","2018-10-11","2019-01-11","2019-04-11","2019-07-11","2019-10-11",
   #               "2020-01-13","2020-04-13","2020-07-13","2020-10-13","2021-01-13"],
   # "fixingRate":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001817],
          "fixingDate":["2021-01-13"],
    "fixingRate":[0.001817],
    "businessDayConvention": "ModifiedFollowing",
    "maturityDateConvention": "ModifiedFollowing",
    "redemption": 100
  },
  "discountCurve": {
    "applyShock":{
           "method": "parallel",
          "shockValues" : [
        {
          "tenor" : "all",
          "shockValue" : 1.0
        }
      ]
    },
    "settlementDays": 2,
    "intpMethod": "LINEAR",
    "currency": "TRY",
    "calendar": "Turkey",
    "dayCounter": "Actual360",
    "period": "6M",
    "instruments": {
      "IRS": {
        "frequency": "Semiannual",
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      },
      "DEPO": {
        "businessDayConvention": "Following",
        "dayCounter": "Actual360"
      }
    }
  },
  "yieldData": yielddata.to_dict('r')
}




In [10]:
try:
    data = api.createTask(CalculationType.FLOATING_RATE_BOND, data3,True)
    result3=json.loads(data["result"])
except RhoovaError as e:
    e.printPretty()

In [11]:
npvTable = pt.PrettyTable(['Parameters', 'Value'])
npvTable.add_row(['PV', result3.get('pv')])
npvTable.add_row(['Clean Price', result3.get('cleanPrice')])
npvTable.add_row(['Dirty Price', result3.get('dirtyPrice')])
npvTable.add_row(['Accrued Amount', result3.get('accruedAmount')])
npvTable.add_row(['Yield to Maturity', 100*result3.get('yieldToMaturity')])
npvTable.add_row(['Duration', result3.get('duration')])
npvTable.add_row(['Modified Duration', result3.get('modifiedDuration')])
npvTable.add_row(['Macualay Duration', result3.get('macaulayDuration')])
npvTable.add_row(['Convexity', result3.get('convexity')])
npvTable.add_row(['Bps', result3.get('bps')])
npvTable.align = 'r'
#npvTable.float_format = '.4'
print(npvTable)

+-------------------+----------------------+
|        Parameters |                Value |
+-------------------+----------------------+
|                PV |    1000.065522969356 |
|       Clean Price |    99.99797201915783 |
|       Dirty Price |   100.00655229693561 |
|    Accrued Amount | 0.008580277777777777 |
| Yield to Maturity |   0.4445164897441867 |
|          Duration |    4.487338252376002 |
| Modified Duration |    4.482357048323748 |
| Macualay Duration |       4.487338252376 |
|         Convexity |    21.28666823753539 |
|               Bps |  0.04516334119753568 |
+-------------------+----------------------+


In [12]:
dur=-1*((result3.get('dirtyPrice')-result2.get('dirtyPrice'))/(2*0.0001))/result.get('dirtyPrice')

dur

0.2027951765896319